In [1]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import mat73
from net import EEG_CNN

2022-01-24 13:17:19.201645: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-24 13:17:19.201674: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
sources = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/downsampled_dipoles-5k/1e-15/sources_20TeD.npy')
eeg = np.load('/media/thanos/Elements/thanos/sim_data/sim_type_1/downsampled_dipoles-5k/1e-15/eeg_20TeD.npy')
fwd = solve_forward()
eeg_topos = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/downsampled_dipoles-5k/1e-15/stand/eeg_20TeD_topos.mat')['eeg_topos']
eeg_topos = eeg_topos.transpose(2, 0, 1)

sim = Simulation(fwd=fwd, source_data=sources, eeg_data=eeg)
eeg_cnn = EEG_CNN(sim=None, eeg_topographies=None)

eeg_cnn.load_nn('/media/thanos/Elements/thanos/nn_trained/Cnn models/eeg_cnn_12.h5')

2022-01-24 13:17:22.136217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-24 13:17:22.136251: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-24 13:17:22.136273: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thanos): /proc/driver/nvidia/version does not exist
2022-01-24 13:17:22.136486: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-24 13:17:22.164118: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 138444800 exceeds 10

Loaded model in EEG_CNN :
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 65, 65, 8)         80        
                                                                 
 flatten (Flatten)           (None, 33800)             0         
                                                                 
 dense (Dense)               (None, 1024)              34612224  
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                              

2022-01-24 13:17:22.898123: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 138444800 exceeds 10% of free system memory.


In [3]:
import scipy.io
topos_eval =  scipy.io.loadmat('./real_data/standardized/eeg_topo_real.mat')['Zi'] 

topos_eval= np.expand_dims(topos_eval, axis=0)

In [4]:
predicted_sources = eeg_cnn.predict(eeg=topos_eval).T

In [ ]:
print(np.min(eeg_topos),np.max(eeg_topos))

In [ ]:
print(np.min(topos_eval),np.max(topos_eval))

In [5]:
np.save('../../Downloads/pred_sources_real.npy',predicted_sources)